In [5]:
import requests, datetime
from pymongo import MongoClient

## Set up
Run the following cells below to set up the backend components

### Start MongoDB Instance

Make sure you have docker running in the background

The command below will start two MongoDB instances.
1. The database where our extracted and transformed data will reside will listening on port 27017
2. The database where our sentiment analyzed data will reside will be listening on port 27018

In [6]:
!docker-compose up -d

[+] Running 0/0
 ⠋ Network news_default  Creating                                          0.1s 
[+] Running 0/1
 ⠙ Network news_default  Creating                                          0.2s 
[+] Running 0/1
 ⠹ Network news_default  Creating                                          0.3s 
[+] Running 1/1
 ✔ Network news_default                  Created                           0.3s 
 ⠋ Container news-news-store-mongo-1     Creating                          0.0s 
 ⠋ Container news-news-sentiment-mong-1  Creating                          0.0s 
[+] Running 1/3
 ✔ Network news_default                  Created                           0.3s 
 ⠙ Container news-news-store-mongo-1     Creating                          0.1s 
 ⠙ Container news-news-sentiment-mong-1  Creating                          0.1s 
[+] Running 1/3
 ✔ Network news_default                  Created                           0.3s 
 ⠹ Container news-news-store-mongo-1     Creating                          0.2s 
 ⠹ Container 

### Start the Flask servers

#### Start the extraction server

Run the cell below, and on the terminal, paste the following:
```bash
python3 -m venv venv
source venv/bin/activate
pip install -r requirements.txt
python3 app.py
```

In [3]:
!open -a Terminal ./extract

#### Start the sentiment server

Run the cell below, and on the terminal, paste the following:
```bash
python3 -m venv venv
source venv/bin/activate
pip install -r requirements.txt
python3 app.py
```

In [4]:
!open -a Terminal ./sentiment-analysis

## Pre-init- Cold Start

**Note**: Ignore this section if you have already run this notebook once. Running this section will erase all your data. This section is only meant
if you are running this notebook for the first time, or if you are testing and want to get rid of the data you inserted previously.

With the servers up and running, we will now run the following cells below to ensure that we are initializing an empty database

In [7]:
#lets start with the extractionDB
client = MongoClient('mongodb://localhost:27017') ## or MongoClient("localhost:27")
db = client.test_db
db.list_collection_names() #this should be empty

['test', 'transformedData']

In [8]:
#drop collection if previous cell wasn't empty
for collection in db.list_collection_names():
    db[collection].drop()

In [9]:
db.list_collection_names() #this should be empty

[]

In [10]:
#lets now repeat for sentimentDB
client = MongoClient('mongodb://localhost:27018') ## or MongoClient("localhost:27")
db = client.test_db
db.list_collection_names() #this should be empty

[]

In [11]:
#drop collection if previous cell wasn't empty
for collection in db.list_collection_names():
    db[collection].drop()
db.list_collection_names() #this should be empty

[]

## Data Populate

This is a cold start procedure where our application is assumed to go live for the first time

### Extract

In [12]:
#make sure the extraction flask app is listening on port 8001
url = 'http://localhost:8001/test'

response = requests.get(url)

print(response.json())

{'test': 'test'}


In [13]:
endpoints = [
    '/',
    '/nyt',
    '/mediastack-gen',
    '/yfinance'
]
for endpt in endpoints:
    url = f"http://localhost:8001"+endpt
    response = requests.get(url)
    print(response.json())

{'message': 'Successfully pushed data to mongo'}
{'message': 'success'}
{'message': 'Successfully pushed data to mongo'}
{'message': 'Successfully pushed data to mongo'}


### Transform

In [14]:
#make sure the sentiment-analysis flask app is listening on port 8002
url = 'http://localhost:8002/transform'

response = requests.get(url)

print(response.json())

{'message': 'Successfully pushed transformed data to mongo'}


### Load

In [15]:
#call endpoint to run sentiment
#make sure the extraction flask app is listening on port 8001
url = 'http://localhost:8002/sentiment'

response = requests.get(url)

print(response.json())

{'message': 'Data push to sentiment DB- Succeded'}


## The Publicly Accesible Data

In [16]:
client = MongoClient('mongodb://localhost:27018') #connect to sentiment db
db = client.test_db
collection = db.test
for doc in collection.find():
    print(doc)

{'_id': ObjectId('64473a8d187fff027843b4c1'), 'date': '04-23-2023', 'data': {'04-23-2023': {'Stock Market': {'avg_sentiment': [], 'news': [{'Headline': 'Bed Bath & Beyond came out of the 2008 downturn a winner. While competitors like Sharper Image and Linens ’n Things filed for bankruptcy, Bed Bath & Beyond actually expanded its business by acquiring other retailers. Its home-goods emporiums full of towels and kitchen aids — all available at a reduced price with that Big Blue coupon — were beacons that kept shoppers coming back.', 'sentiment': {'positive': 0.44310519099235535, 'negative': 0.022097798064351082, 'neutral': 0.5347970128059387}, 'source': 'nyt'}]}, 'IBM': {'avg_sentiment': [], 'news': [{'Headline': 'This week in tech: Shares slide at Netflix, Tesla, Alphabet | Pro Recap', 'sentiment': {'positive': 0.007768487557768822, 'negative': 0.9471878409385681, 'neutral': 0.045043740421533585}, 'source': 'yfinance'}]}, 'Verizon Communications': {'avg_sentiment': [], 'news': [{'Headli

## Insert Historical Data

Up until the previous section, we only inserted data with the assumption of a cold start, aka, we are launching our application for the first time. This section is to populate the whole
database with past news data. More to be added soon.

## Stopping Docker Mongo Instances

In [1]:
!docker-compose down

[+] Running 0/0
 ⠋ Container news-news-sentiment-mong-1  Stopping                          0.1s 
 ⠋ Container news-news-store-mongo-1     Stopping                          0.1s 
[+] Running 0/2
 ⠙ Container news-news-sentiment-mong-1  Stopping                          0.2s 
 ⠙ Container news-news-store-mongo-1     Stopping                          0.2s 
[+] Running 0/2
 ⠹ Container news-news-sentiment-mong-1  Stopping                          0.3s 
 ⠹ Container news-news-store-mongo-1     Stopping                          0.3s 
[+] Running 0/2
 ⠸ Container news-news-sentiment-mong-1  Stopping                          0.4s 
 ⠸ Container news-news-store-mongo-1     Stopping                          0.4s 
[+] Running 0/2
 ⠼ Container news-news-sentiment-mong-1  Stopping                          0.5s 
 ⠼ Container news-news-store-mongo-1     Stopping                          0.5s 
[+] Running 0/2
 ⠴ Container news-news-sentiment-mong-1  Stopping                          0.6s 
 ⠴ Container 